# Configuring cloud-masking in Sentinel-2

Following this tutorial https://eemont.readthedocs.io/en/0.3.5/tutorials/001-Clouds-and-Shadows-Masking-Sentinel-2.html

## Description

The default cloud mask for Sentinel-2 can be poor. `eemont` implements [s2Cloudless](https://github.com/sentinel-hub/sentinel2-cloud-detector) to improve the cloudmasking of S2, and it can be configured to make the mask more or less aggressive depending on your requirements. Some examples of configuring aspect of the cloud mask are shown below.

## Load packages

Import Python packages that are used for the analysis.


In [ ]:
%matplotlib inline

import geemap as gmap
import ee
import eemont

### Connect to Google Earth Engine (GEE)

In [ ]:
m = gmap.Map()

***

## Load Sentinel 2 image collection using eemont

Only `S2_SR` works, the `S2_SR_HARMONIZED` product apparently hasn't been incorporated into eemont's library yet.

> So **don't load data after 25th Jan 2022 if using `S2_SR` as the scale offset before/after this date is different**


In [ ]:
Map = gmap.Map(center=[-35.2041, 149.2721], zoom=9)

# Create a point with set coordinates (canberra parliament)
point = ee.Geometry.Point([149.1244, -35.3096])
    
#load the S2 product
S2 = (ee.ImageCollection('COPERNICUS/S2_SR')
                  .filterBounds(point) # filter to some region
                  .closest('2018-12-15', tolerance=2, unit='week') # find image closest to this date
                  .scale() #note this step!
     )

## Configure the cloud mask

Because above we `scale()` the data, we need to supply the `scaledImage=True` parameter in the args below.

In [ ]:
#increase/decrease probability of detected cloud
S2a = S2.maskClouds(prob=75, scaledImage=True)
S2aa = S2.maskClouds(prob=45, scaledImage=True)

#increase buffer around clouds to get more shadows
S2b = S2.maskClouds(buffer=500, scaledImage=True)

#Cloud Probability masking with a different NIR threshold
# By default, NIR values under 0.15 are considered potential cloud shadows.
# This value can be changed by modifying the dark parameter.
S2c = S2.maskClouds(dark = 0.1, scaledImage=True)

# Cloud Probability masking using the Cloud Displacement Index (CDI)
# CDI is an index used to avoid masking bright surfaces as clouds.
# By default, CDI is not used, but if required, the CDI value can be changed by modifying the cdi parameter.
# CDI values go from -1 to 1. The most used value for cloud masking is -0.5.
S2d = S2.maskClouds(cdi = -0.5, scaledImage=True)

## Plot

In [ ]:
#after scaling, numbers are 0-1
vis_scaled = {
  'min': 0.0,
  'max': 0.3,
  'bands': ['B4', 'B3', 'B2'],
};

# add just the first image in the collection to the map
Map.addLayer(S2, vis_scaled, 'S2 no cloud-masking')
Map.addLayer(S2a, vis_scaled, 'S2 75% threshold')
Map.addLayer(S2aa, vis_scaled, 'S2 45% threshold')
Map.addLayer(S2b, vis_scaled, '500m buffer')
Map.addLayer(S2c, vis_scaled, 'Dark 0.1')
Map.addLayer(S2d, vis_scaled, 'CDI=-0.5')
Map.addLayerControl()
Map

***